In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from itertools import chain
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import numpy as np
import re
from numpy import random
from datetime import date
import sqlalchemy as sa

In [140]:
#codes=pd.read_excel("C:/Users/homa/Python-Notbook/TradeMap/Product Code.xlsx",converters={'HS Code':str})


In [3]:
codes=pd.read_excel("C:/Users/homa/Python-Notbook/TradeMap/CODES.xlsx")


if len (codes['HS Code'])>6:
   codes['HS Code'] = codes['HS Code'].astype(str)
   codes['HS Code6']="'" + codes['HS Code'].str[:6]

else:
   codes['HS Code'] = codes['HS Code'].astype(str)
   codes['HS Code6']="'" +codes['HS Code']



Product_Code=codes.drop_duplicates(subset=['HS Code6'])


In [4]:
Product_Code.head(20)

,HS Code,Product,factory,E_group,p_group,HS Code6
0,16041400,کنسرو تن ماهی,cahshni,Tunas,تن ماهی,'160414
1,17041000,آدامس شیک,pars minoo,Chewing gum,آدامس,'170410
2,17049000,شکلات قهوه,pars minoo,Sugar confectionery other than chewing gum,شیرینی غیر از آدامس,'170490
5,18069000,شکلات میکا,choco pars,Chocolate & other food preparations containing...,شکلات,'180690
6,19041000,اسنک نمکی مینوشا,sanati minoo,Prepared foods obt. by the swelling/roasting o...,اسنک,'190410
7,19053100,بیسکوئیت,minoo shargh,Sweet biscuits,بیسکوئیت,'190531
9,19053200,ویفر شکلاتی نانو,sanati minoo,Waffles & wafers,ویفر,'190532
10,19059090,کیک مغزدار,sanati minoo,"Bread, pastry, cakes, biscuits & other bakers'...",کیک,'190590
11,20011000,خیار شور,cahshni,"Cucumbers & gherkins, prepared by vinegar",خیارشور,'200110
12,20019000,ترشی لیمو,chashni,"Vegetables, fruit, nuts by vinegar",ترشی,'200190


In [5]:
Product_Code['HS Code6']

0     '160414
1     '170410
2     '170490
5     '180690
6     '190410
7     '190531
9     '190532
10    '190590
11    '200110
12    '200190
13    '200290
14    '200490
15    '200600
16    '200710
18    '200899
19    '210690
20    '220210
21    '040900
22    '071021
23    '071120
31    '200599
Name: HS Code6, dtype: object

In [6]:
Product_Code.rename(columns= {'HS Code6':'ProductCode'}, inplace= True)

Product_Code=Product_Code.reset_index(drop=True)

C:\Users\homa\AppData\Local\Temp\ipykernel_31184\2574148061.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Product_Code.rename(columns= {'HS Code6':'ProductCode'}, inplace= True)


In [7]:
driver= webdriver.Chrome()
driver.get("https://www.trademap.org")
#driver.maximize_window()
waits= WebDriverWait(driver, 30)

In [8]:
def login():
    try:
        
        logintext=driver.find_element(By.CSS_SELECTOR ,("#ctl00_MenuControl_Label_Login"))
        if logintext.text=='Login': 
                
            login=waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Login')))
            login.click()
            url= driver.current_url
            if url!='https://www.trademap.org/Index.aspx': 
                getemail= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#Username')))
                getemail.send_keys('homa.abedinzadeh@gmail.com')
                #time.sleep(random.randint(5,20))
                getpassword= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#Password')))
                getpassword.send_keys('Aa@97019701')
                #time.sleep(random.randint(5,20))
                waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.container > div > div:nth-child(2) > div > div > div > form > fieldset > div:nth-child(4) > div > button'))).click()
            
         
    except Exception as e:
                print(f"Error selecting option : {e}")


In [9]:
def delete():
    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Image_deleteProduct'))).click()
    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Image_DeleteCountry'))).click()  

In [15]:



if ('df' in locals() or 'df' in globals()) and len(df)>1:
    
    df= df[df['productcode'] !=  df['productcode'].iloc[-1]]
    Product_Code=Product_Code.iloc[len (df['productcode'].unique()):]
else: 
    df = pd.DataFrame()
    temp = pd.DataFrame()


for kind in range(2):
    kind+=1
    if kind==1:
        time.sleep(random.randint(5,8))
        waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_label_RadioButton_TradeType_Export'))).click()
    for i in range(len(Product_Code)):
        
        login()
        if kind==1:
           waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_label_RadioButton_TradeType_Export'))).click()  

        time.sleep(random.randint(10,15))
        delete()
        time.sleep(random.randint(4,7))
        inputproduct = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_Input')))
        inputproduct.send_keys(Product_Code['ProductCode'].iloc[i])
        time.sleep(random.randint(5,8))
        proclick = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_c0'))).click()
        time.sleep(random.randint(6,9))
        element = driver.find_element(By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators')
        if (element.is_enabled()):
            btn = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators'))).click()
            
            logintext=driver.find_element(By.CSS_SELECTOR ,("#ctl00_MenuControl_Label_Login"))
            if logintext.text=='Login':  
                login()
                if kind==1:
                    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_label_RadioButton_TradeType_Export'))).click()  


            dropdown_element = waits.until(EC.visibility_of_element_located((By.ID, 'ctl00_NavigationControl_DropDownList_Country')))  # Replace with actual ID     
            select = Select(dropdown_element)
            options = select.options
            j=0
            while True:  
                print(j)
                try:
                    
                    dropdown_element = waits.until(EC.visibility_of_element_located((By.ID, 'ctl00_NavigationControl_DropDownList_Country')))
                    select = Select(dropdown_element)
                    options = select.options
                    option_text = options[j].text
                    
                    if  (option_text != 'All') and ('not elsewhere specified' not in option_text):
                        select.select_by_visible_text(option_text)
                        time.sleep(random.randint(2,3))
                        # تغییر تعداد رکوردها به 300
                        incshow = waits.until(EC.element_to_be_clickable((By.XPATH, "//select[@id='ctl00_PageContent_GridViewPanelControl_DropDownList_PageSize']")))
                        
                        drop = Select(incshow)
                        
                        time.sleep(random.randint(2,3))
                        drop.select_by_value('300')




                    
                    # استخراج جدول
                        all_tables = pd.read_html(driver.page_source, attrs={'id': 'ctl00_PageContent_MyGridView1'})
                        temp = all_tables[0]

                        if temp.shape[0] != 1:
                            temp = temp.iloc[1:, 1:].iloc[:, :-4]
                            new_header = temp.iloc[0]
                            temp = temp[1:]
                            temp.columns = new_header

                            # افزودن اطلاعات کشور و کد محصول
                            temp['Source'] = option_text
                            temp['productcode'] = re.sub(r'[^\w]', '', Product_Code['ProductCode'].iloc[i])
                            temp['Year']=int (temp.columns[4][-4:])
                            temp['kind']=kind   #0== import , 1==export
                            
                            temp['insertdate']=date.today()

                            # تغییر نام ستون‌ها
                            average = temp.columns[15]
                            sharein = temp.columns[3]
                            temp.rename (columns={temp.columns[0]:'Destination',temp.columns[1]:'Value_imported' , temp.columns[2]:'Trade_balance',temp.columns[4]:'Quantity_imported',temp.columns[7]:'Growth5y_in_imported_value%'
                            ,temp.columns[8]:'Growth5y_in_imported_quantity%', temp.columns[9]:'Growth1y_in_imported_value%' , temp.columns[10]:'Export_partners_ranking', temp.columns[11]:'Export_share%',
                                temp.columns[12]:' Partner_export_growth5y%', temp.columns[13]:'Average_distance',  temp.columns[14]:'Concentration',average: 'Average tariff%', sharein: 'ShareinCounryImports' }, inplace=True)

                            
                            # افزودن به DataFrame اصلی
                            df = pd.concat([df, temp], axis=0)
                            temp = pd.DataFrame()
                            df.drop(df[df['Destination'] == 'Total'].index, inplace=True)
                            df.drop(df[df['Destination'] == 'World'].index, inplace=True)
                            print(f"Selected option: {option_text}, {j}")
                        time.sleep(random.randint(1,3))
                    if (j+1)%10 ==0:
                            #driver.quit()
                            #driver= webdriver.Chrome()
                            waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Menu_Home'))).click()
                            time.sleep(random.randint(5,8))
                            login()
                            if kind==1:
                                waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_label_RadioButton_TradeType_Export'))).click()  
                            delete()
                            time.sleep(random.randint(4,7))
                            inputproduct = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_Input')))
                            inputproduct.send_keys(Product_Code['ProductCode'].iloc[i])
                            time.sleep(random.randint(5,8))
                            proclick = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_c0'))).click()
                            time.sleep(random.randint(6,9))
                            btn = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators'))).click()
                            
                
                except Exception as e:
                    print(f"Error selecting option {j}: {e}")
                    site = driver.current_url
                    if site!= 'https://www.trademap.org/Index.aspx':
                        driver.get('https://www.trademap.org/Index.aspx')
                    login()
                    if kind==1:
                        waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_label_RadioButton_TradeType_Export'))).click()  
                    delete()
                    time.sleep(random.randint(4,7))
                    inputproduct = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_Input')))
                    inputproduct.send_keys(Product_Code['ProductCode'].iloc[i])
                    time.sleep(random.randint(5,8))
                    proclick = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_c0'))).click()
                    time.sleep(random.randint(6,9))
                    btn = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators'))).click()
                    if len (df[(df.Source==option_text) & (df.productcode==re.sub(r'[^\w]', '', Product_Code['ProductCode'].iloc[i])) ])==0:
                        j-=1
                    
                    
                
                j+=1
                if j+1> len (options):
                    
                    print (f"Get All Country this product:{Product_Code['ProductCode'].iloc[i]}")
                    break
        waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Menu_Home'))).click()
        delete()
        time.sleep(30)
        #driver.refresh()
        
        driver.get('https://www.trademap.org/Index.aspx')
        i+=1
    print ('All Requests Import')
    driver.get('https://www.trademap.org/Index.aspx')
    kind+=1
print('All Requests Export')

0
1
Selected option: Afghanistan, 1
2
3
Selected option: Albania, 3
4
Selected option: Algeria, 4
5
6
7
Selected option: Andorra, 7
8
Selected option: Angola, 8
9
Selected option: Anguilla, 9
10
Selected option: Antigua and Barbuda, 10
11
Selected option: Area Nes, 11
12
Selected option: Argentina, 12
13
Selected option: Armenia, 13
14
Selected option: Aruba, 14
15
16
Selected option: Australia, 16
17
Selected option: Austria, 17
18
Selected option: Azerbaijan, 18
19
Selected option: Bahamas, 19
20
Error selecting option 20: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6D6BF6CB5+28821]
	(No symbol) [0x00007FF6D6B63840]
	(No symbol) [0x00007FF6D6A0578A]
	(No symbol) [0x00007FF6D6A591BE]
	(No symbol) [0x00007FF6D6A594AC]
	(No symbol) [0x00007FF6D6AA2647]
	(No symbol) [0x00007FF6D6A7F33F]
	(No symbol) [0x00007FF6D6A9F412]
	(No symbol) [0x00007FF6D6A7F0A3]
	(No symbol) [0x00007FF6D6A4A778]
	(No symbol) [0x00007FF6D6A4B8E1]
	GetHandleVerifier [0x00007FF6D6F2FCAD+3408013]
	GetHandleVer

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6D6BF6CB5+28821]
	(No symbol) [0x00007FF6D6B63840]
	(No symbol) [0x00007FF6D6A0578A]
	(No symbol) [0x00007FF6D6A591BE]
	(No symbol) [0x00007FF6D6A594AC]
	(No symbol) [0x00007FF6D6AA2647]
	(No symbol) [0x00007FF6D6A7F33F]
	(No symbol) [0x00007FF6D6A9F412]
	(No symbol) [0x00007FF6D6A7F0A3]
	(No symbol) [0x00007FF6D6A4A778]
	(No symbol) [0x00007FF6D6A4B8E1]
	GetHandleVerifier [0x00007FF6D6F2FCAD+3408013]
	GetHandleVerifier [0x00007FF6D6F4741F+3504127]
	GetHandleVerifier [0x00007FF6D6F3B5FD+3455453]
	GetHandleVerifier [0x00007FF6D6CBBDBB+835995]
	(No symbol) [0x00007FF6D6B6EB5F]
	(No symbol) [0x00007FF6D6B6A814]
	(No symbol) [0x00007FF6D6B6A9AD]
	(No symbol) [0x00007FF6D6B5A199]
	BaseThreadInitThunk [0x00007FF9BE35259D+29]
	RtlUserThreadStart [0x00007FF9BE5AAF38+40]


In [37]:
df.head()

1,Destination,Value_imported,Trade_balance,ShareinCounryImports,Quantity_imported,Quantity unit,Unit value (USD/unit),Growth5y_in_imported_value%,Growth5y_in_imported_quantity%,Growth1y_in_imported_value%,...,Export_share%,Partner_export_growth5y%,Average_distance,Concentration,Average tariff%,Source,productcode,Year,kind,insertdate
3,Czech Republic,33,33,100,4,Tons,8250,NaN,NaN,NaN,...,0.6,4,3753,0.18,0,Afghanistan,160414,2023,1,2024-11-13
4,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.5,1,11234,0.23,0.9,Afghanistan,160414,2023,1,2024-11-13
5,Italy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.1,7,5984,0.2,0,Afghanistan,160414,2023,1,2024-11-13
6,Spain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.3,6,8883,0.17,0,Afghanistan,160414,2023,1,2024-11-13
7,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.8,3,5346,0.15,0,Afghanistan,160414,2023,1,2024-11-13


In [37]:
df.shape

(622185, 19)

In [38]:
df.drop_duplicates(subset=['Exporters', 'Importer', 'productcode'], inplace=True)

In [39]:
df.shape

(622185, 19)

In [41]:
connection_string = (
    'Driver=ODBC Driver 17 for SQL Server;'
    'Server=DESKTOP-7TRDV4C;'
    'Database=trademap;'
    'UID=sa;'
    'PWD=Aa@97019701;'
    'Trusted_Connection=no;'
)
connection_url = sa.engine.URL.create(
    "mssql+pyodbc", 
    query=dict(odbc_connect=connection_string)
)
engine = sa.create_engine(connection_url, fast_executemany=True)

# Deleting existing data in SQL Table:-


# upload the DataFrame
df.to_sql("total", engine, schema="dbo", if_exists="append", index=False)
df = pd.DataFrame()

In [26]:
all_videos_title = []
all_videos = []
all_videos =driver.find_element(By.CLASS_NAME,"radioSwitch")
for sl_video in range(2):
    print (sl_video)


0
1


In [27]:
waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_label_RadioButton_TradeType_Export'))).click()